In [3]:
# TODO: refactor this into utility

import random
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import requests 
import numpy as np
from datetime import datetime, date, timedelta

class DataGenerator:
    def __init__(self,num_docs=10,num_patients=100,min_notes_per_discharge=5,max_notes_per_discharge=10,
                 max_stay_per_patient=6,start=None,end=None):
        self.MIN_NOTES_PER_DIS = min_notes_per_discharge
        self.MAX_NOTES_PER_DIS = max_notes_per_discharge
        self.MAX_STAY_PER_PATIENT = max_stay_per_patient     
        self.drs = [name for name in self._get_random_names(num_docs)]
        self.drs_non_collab = [name for name in self._get_random_names(num_docs)]
        self.patients = [name for name in self._get_random_names(num_patients, "patient")]    
        self.end = datetime.now() if end == None else end
        self.start = datetime(self.end.year-1,self.end.month,self.end.day) if start == None else start
        self._create_date_range()
        self._generate_data()

    def _get_random_names(self,count, prefix="dr"):
#         response = requests.get(f'https://randomuser.me/api?results={count}&inc=name&nat=us,gb,es')
#         names = [f"{user['name']['first']} {user['name']['last']}" for user in response.json()['results']]
        names = [f"{prefix}{i+1}" for i in range(0,count)]
        return names

    def _get_date(self):
        return random.choice(self.date_range)

    def _generate_notes_for_discharge(self, discharge):
#         disposition = discharge[4]
#         dr_list = self.drs if disposition < 1 else self.drs_non_collab
        dr_list = random.choices(self.drs,k=random.randint(6,10))
        notes = [(discharge[0], random.choice(dr_list), discharge[2] + self._get_random_timedelta()) 
                 for _ in range(0, random.randint(self.MIN_NOTES_PER_DIS,self.MAX_NOTES_PER_DIS))]
        return notes

    def _create_date_range(self):
        dr = np.arange(self.start, self.end, np.timedelta64(1, 'D'))
        self.date_range = [date for date in dr.astype(object)]

    def _generate_notes(self,discharges):
        for d in discharges:
            yield self._generate_notes_for_discharge(d)

    def _get_random_timedelta(self):
        return timedelta(days=random.randint(1,self.MAX_STAY_PER_PATIENT))
    
    def _flip(self):
        return 0 if random.random() > .1 else 1

    def _generate_data(self):  
        discharges = [(i, p, d := self._get_date(), d + timedelta(days=self.MAX_STAY_PER_PATIENT), self._flip(), random.randint(65,80))
            for i,p in enumerate(self.patients)]

        notes = list(chain.from_iterable(self._generate_notes(discharges)))
        
        dis_columns = ['id', 'patient', 'arrive_date', 'discharge_date', 'disposition', 'age']

        self.dis_df = pd.DataFrame(discharges, columns=dis_columns)

        note_columns=['discharge_id', 'dr', 'date']
        
        self.note_df = pd.DataFrame(notes, columns=note_columns)
    
if __name__ == "__main__":
    data = DataGenerator(num_docs=300, num_patients=700, max_stay_per_patient=7,min_notes_per_discharge=150,max_notes_per_discharge=180)
    print(data.note_df.head())
    print(data.note_df.shape)
    print(data.dis_df.head())
    print(data.dis_df.shape)
    
    data.note_df.to_csv('../data/notes_w_disposition_abdul_test.csv', index=False)
    data.dis_df.to_csv('../data/discharges_w_disposition_abdul_test.csv', index=False)



   discharge_id     dr       date
0             0  dr180 2020-06-06
1             0   dr64 2020-06-11
2             0  dr180 2020-06-11
3             0  dr187 2020-06-08
4             0  dr214 2020-06-10
(115574, 3)
   id   patient arrive_date discharge_date  disposition  age
0   0  patient1  2020-06-05     2020-06-12            0   68
1   1  patient2  2020-12-01     2020-12-08            0   69
2   2  patient3  2021-02-06     2021-02-13            0   80
3   3  patient4  2020-08-12     2020-08-19            1   77
4   4  patient5  2020-08-05     2020-08-12            1   75
(700, 6)


In [24]:
def get_random_names(count, prefix="dr"):
    names = [f"{prefix}{i}" for i in range(1,count)]
    return names

names = get_random_names(10)

choices = random.choices(names,k=5)

print(choices)

['dr3', 'dr5', 'dr6', 'dr3', 'dr4']


In [5]:
l = []
l.append({'id':1,'name':'grant'})
l.append({'id':1,'name':'henry'})
l_df = pd.DataFrame.from_dict(l)
l_df.head()

,id,name
0,1,grant
1,1,henry
